In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [25]:
import cudf 
import cupy
import cuml

In [26]:
train = cudf.read_csv('/kaggle/input/ipl-2020-player-performance/Training.csv')
test = cudf.read_csv('/kaggle/input/ipl-2020-player-performance/sample_submission.csv')

pre= cudf.read_csv('/kaggle/input/ipl-2020-player-performance/Matches IPL 2008-2019.csv')
coming =cudf.read_csv('/kaggle/input/ipl-2020-player-performance/Matches IPL 2020.csv')



In [27]:
print('shape of train:', train.shape)
print('shape of test:', test.shape)
print('shape of pre:', pre.shape)
print('shape of coming:', coming.shape)

In [28]:
train.head()

In [29]:
train['player'] = train['Id'].copy(deep=True)
train['match_id'] =  train['Id'].copy(deep=True)
for i in range(0, len( train)):
    ss = train['Id'][i].split("_")
    train['player'][i] =  ss[-1]
    train['match_id'][i] = int(ss[0])

In [30]:
train.head()

In [31]:
train_df=train[['Id', 'player', 'Total Points', 'match_id']]
train_df.head()

In [32]:
test.head()

In [33]:
test['player'] = test['Id'].copy(deep=True)
test['match_id'] =  test['Id'].copy(deep=True)
for i in range(0, len( test)):
    ss = test['Id'][i].split("_")
    test['player'][i] =  ss[-1]
    test['match_id'][i] = int( ss[0])

In [34]:
test.head()

In [35]:
pre.head()

In [36]:
pre=pre[['match_id', 'team1', 'team2', 'season']]
pre.head()

In [37]:
coming.head()

In [38]:
coming=coming[['match_id', 'team1', 'team2']]
coming.head()

In [39]:
train_df = cudf.merge(train_df,pre, how = 'inner', on = 'match_id')
train_df.head()

In [40]:
test = cudf.merge(test,coming, how = 'inner', on = 'match_id')
test.head()

In [41]:
print('train_df shape:', train_df.shape)
print('test shape', test.shape)

In [42]:
train_df.isnull().sum()

In [43]:
test.isnull().sum()

In [44]:
print(train_df.info())
print(test.info())

In [45]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
#Using pandas and numpy

In [47]:
train = pd.read_csv('/kaggle/input/ipl-2020-player-performance/Training.csv')
test = pd.read_csv('/kaggle/input/ipl-2020-player-performance/sample_submission.csv')

pre= pd.read_csv('/kaggle/input/ipl-2020-player-performance/Matches IPL 2008-2019.csv')
coming =pd.read_csv('/kaggle/input/ipl-2020-player-performance/Matches IPL 2020.csv')

In [48]:
train['player'] = train['Id']
train['match_id'] =  train['Id']
for i in range(0, len( train)):
    train['player'][i] =  train['Id'][i].split("_")[-1]
    train['match_id'][i] = int( train['Id'][i].split('_')[:1][0])

In [49]:
train_df=train[['Id', 'player', 'Total Points', 'match_id']]
train_df.head()

In [50]:
test['player'] = test['Id']
test['match_id'] =  test['Id']
for i in range(0, len( test)):
    test['player'][i] =  test['Id'][i].split("_")[-1]
    test['match_id'][i] = int( test['Id'][i].split('_')[:1][0])
    

In [51]:
pre=pre[['match_id', 'team1', 'team2', 'season']]
pre.head()

In [52]:
coming=coming[['match_id', 'team1', 'team2']]
coming.head()

In [53]:
train_df = pd.merge(train_df,pre, how = 'inner', on = 'match_id')
train_df.head()

In [54]:
test = pd.merge(test,coming, how = 'inner', on = 'match_id')
test.head()

In [56]:
print('train_df shape:', train_df.shape)
print('test shape', test.shape)

In [57]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [58]:
train_df.team1 = le.fit_transform(train_df.team1)
train_df.team2 = le.fit_transform(train_df.team2)

In [59]:
X=train_df[['team1', 'team2']]
y=train_df['Total Points']

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV ,RandomizedSearchCV
from sklearn.tree import plot_tree


from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR


In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=15)

In [71]:
#import models
lg = LinearRegression()
svr = SVR()
dt = DecisionTreeRegressor( criterion="friedman_mse", splitter= 'random')
rf = RandomForestRegressor()
ada = AdaBoostRegressor()
gb = GradientBoostingRegressor()


#fit model in train data
lg.fit(X_train,y_train)
svr.fit(X_train,y_train)
dt.fit(X_train,y_train)
rf.fit(X_train,y_train)
ada.fit(X_train,y_train)
gb.fit(X_train,y_train)



#predict test data
lg_pred = lg.predict(X_test) 
svr_pred = svr.predict(X_test) 
dt_pred = dt.predict(X_test) 
rf_pred = rf.predict(X_test) 
ada_pred = ada.predict(X_test) 
gb_pred = gb.predict(X_test) 



# measure the errors
lg_RMSE = np.sqrt(mean_squared_error(y_test,lg_pred))
svr_RMSE = np.sqrt(mean_squared_error(y_test,svr_pred))
dt_RMSE = np.sqrt(mean_squared_error(y_test,dt_pred))
rf_RMSE = np.sqrt(mean_squared_error(y_test,rf_pred))
ada_RMSE = np.sqrt(mean_squared_error(y_test,ada_pred))
gb_RMSE = np.sqrt(mean_squared_error(y_test,gb_pred))



#rmse of models
print('lg_RMSE',lg_RMSE)
print('svr_RMSE',svr_RMSE)
print('dt_RMSE',dt_RMSE)
print('rf_RMSE',rf_RMSE)
print('ada_RMSE',ada_RMSE)
print('gb_RMSE',gb_RMSE)

In [72]:
test.team1 = le.fit_transform(test.team1)
test.team2 = le.fit_transform(test.team2)

In [73]:
test.head()

In [74]:
df = train_df.append(test, ignore_index=True)

In [75]:
df.head()

In [76]:
df.tail()

In [77]:
train.shape

In [78]:
df_Train=df.iloc[:15916,:]
df_Test=df.iloc[15916:,:]

In [79]:
df_Train.head()

In [80]:
df_Train.drop(['Id', 'player', 'match_id', 'season'],axis=1,inplace=True)

In [81]:
df_Train.head()

In [82]:
df_Test.head()

In [83]:
df_Test.drop(['Id', 'player', 'Total Points', 'match_id', 'season'],axis=1,inplace=True)

In [84]:
df_Test.head()

In [85]:
X_train=df_Train.drop(['Total Points'],axis=1)
y_train=df_Train['Total Points']

In [86]:
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor(n_estimators=100, max_depth=4)

In [87]:
GBR.fit(X_train, y_train)

In [88]:
y_pred=GBR.predict(df_Test)

In [89]:
y_pred.shape

In [90]:
y_pred

In [91]:
test.head()

In [92]:
##Create Sample Submission file and Submit
sub=pd.DataFrame(y_pred)
submission=pd.concat([test['Id'],sub],axis=1)
submission.columns=['Id','Total Points']

In [93]:
submission.to_csv('submission.csv',index=False)

In [94]:
submission.tail()